In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pathlib
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import DataTrustedIdentifier, Image, create_info_image
from sqlalchemy import and_
from sqlalchemy.orm import Session

## Database connection

In [3]:
session: Session
engine, session = db.connect()
engine.echo=False

2023-03-24 20:46:33,601 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 20:46:33,602 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 20:46:33,603 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 20:46:33,604 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 20:46:33,604 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 20:46:33,605 INFO sqlalchemy.engine.Engine [raw sql] {}


## Create table info_image and fill table

In [ ]:
def fill_table_info_image(color_mode, image_size, result, seq):
    for r in result:
        record_exist = session.query(Image) \
            .filter(and_(Image.color_mode == color_mode,
                         Image.image_size == image_size,
                         Image.path_image == r,
                         Image.seq_id == seq)) \
            .all()
        if len(record_exist) == 0:
            print('inserted new record')
            image_info = create_info_image(color_mode, image_size, r, seq)
            session.add(image_info)
            session.commit()


def set_path_image(color_mode, image_size, list_images, query):
    for i, q in enumerate(query):
        result = sorted([str(image) for image in list_images if q.barcode in str(image.stem) and os.path.exists(image)])
        print('founded %d files' % len(result))
        if len(result) > 0:
            fill_table_info_image(color_mode, image_size, result, q.seq)

In [ ]:
db.create_table(engine, Image)

for color in ['rgb', 'grayscale']:
    for image_size in ['256', '400', '512']:
        column_name = 'path_%s_%s' % (image_size, color)
        path_fotos = '/home/xandao/Documentos/%s/%s/w_pred_mask' % (color.upper(), image_size)
        list_images = [image for image in pathlib.Path(path_fotos).rglob('*.jpeg') if image.is_file()]
        columns = [DataTrustedIdentifier.barcode, DataTrustedIdentifier.seq]
        query = session.query(*columns).all()
        if len(list_images) > 0 and len(query) >= 0:
            set_path_image(color, image_size, list_images, query)

## Close connection

In [5]:
session.close()
engine.dispose()